In [1]:
import pandas as pd

In [2]:
## 2016 Batters ##
batters_L_2016 = pd.read_csv("batter_splits/batters_vL_2016.csv")
batters_R_2016 = pd.read_csv("batter_splits/batters_vR_2016.csv")
## 2017 Batters ##
batters_L_2017 = pd.read_csv("batter_splits/batters_vL_2017.csv")
batters_R_2017 = pd.read_csv("batter_splits/batters_vR_2017.csv")
## 2018 Batters ##
batters_L_2018 = pd.read_csv("batter_splits/batters_vL_2018.csv")
batters_R_2018 = pd.read_csv("batter_splits/batters_vR_2018.csv")
## 2016 Pitchers ##
pitchers_L_2016 = pd.read_csv("pitcher_splits/pitcher_vL_2016.csv")
pitchers_R_2016 = pd.read_csv("pitcher_splits/pitcher_vR_2016.csv")
## 2017 Pitchers ##
pitchers_L_2017 = pd.read_csv("pitcher_splits/pitcher_vL_2017.csv")
pitchers_R_2017 = pd.read_csv("pitcher_splits/pitcher_vR_2017.csv")
## 2018 Pitchers ##
pitchers_L_2018 = pd.read_csv("pitcher_splits/pitcher_vL_2018.csv")
pitchers_R_2018 = pd.read_csv("pitcher_splits/pitcher_vR_2018.csv")

salary = pd.read_csv("salaries/20190402.csv")
playeridmap = pd.read_csv("PLAYERIDMAP.csv")

In [3]:
pitcher_salary = salary[salary['Position'] == "SP"]
batter_salary = salary[salary['Roster Position'] != "P"]

In [4]:
pitcher_woba_2018 = pd.merge(pitchers_L_2018, pitchers_R_2018,
                        left_on="playerid", right_on="playerid", 
                        suffixes=('_vL', '_vR'))
pitcher_woba_2017 = pd.merge(pitchers_L_2017, pitchers_R_2017,
                        left_on="playerid", right_on="playerid",
                        suffixes=('_vL', '_vR'))
pitcher_woba_2016 = pd.merge(pitchers_L_2016, pitchers_R_2016,
                        left_on="playerid", right_on="playerid",
                        suffixes=('_vL', '_vR'))
pitcher_woba_2016.rename(index=str, columns={'Name_vR':'Name', 'Team_vR':'Team'}, inplace=True)
pitcher_woba_2017.rename(index=str, columns={'Name_vR':'Name', 'Team_vR':'Team'}, inplace=True)
pitcher_woba_2018.rename(index=str, columns={'Name_vR':'Name', 'Team_vR':'Team'}, inplace=True)
pitcher_woba_2016.drop(['Name_vL', 'Team_vL'], axis=1, inplace=True)
pitcher_woba_2017.drop(['Name_vL', 'Team_vL'], axis=1, inplace=True)
pitcher_woba_2018.drop(['Name_vL', 'Team_vL'], axis=1, inplace=True)

In [5]:
batter_woba_2018 = pd.merge(batters_L_2018, batters_R_2018,
                            left_on="playerid", right_on="playerid",
                            suffixes=('_vL', '_vR'))
batter_woba_2017 = pd.merge(batters_L_2017, batters_R_2017,
                            left_on="playerid", right_on="playerid",
                            suffixes=('_vL', '_vR'))
batter_woba_2016 = pd.merge(batters_L_2016, batters_R_2016,
                            left_on="playerid", right_on="playerid",
                            suffixes=('_vL', '_vR'))
batter_woba_2016.rename(index=str, columns={'Name_vR':'Name', 'Team_vR':'Team'}, inplace=True)
batter_woba_2017.rename(index=str, columns={'Name_vR':'Name', 'Team_vR':'Team'}, inplace=True)
batter_woba_2018.rename(index=str, columns={'Name_vR':'Name', 'Team_vR':'Team'}, inplace=True)
batter_woba_2016.drop(['Name_vL', 'Team_vL'], axis=1, inplace=True)
batter_woba_2017.drop(['Name_vL', 'Team_vL'], axis=1, inplace=True)
batter_woba_2018.drop(['Name_vL', 'Team_vL'], axis=1, inplace=True)

In [6]:
# Match salary info with fangraphs id
pid_map = playeridmap[['IDFANGRAPHS', 'DRAFTKINGSNAME']].copy()
pid_map.dropna(inplace=True, how="any")
salary_w_id = pd.merge(salary, pid_map, left_on="Name", right_on="DRAFTKINGSNAME")
salary_info = salary_w_id[['Name', 'Roster Position', 'Salary', 'TeamAbbrev', 'IDFANGRAPHS']].copy()

In [37]:
df = pd.DataFrame(columns=['id', 'wRC+_vL', 'wRC+_vR'])

In [49]:
for index, row in salary_info.iterrows():
    dict_woba = {}
    pid = int(row['IDFANGRAPHS'])
    pid_woba_2016 = batter_woba_2016[batter_woba_2016['playerid'] == pid]
    pid_woba_2017 = batter_woba_2017[batter_woba_2017['playerid'] == pid]
    pid_woba_2018 = batter_woba_2018[batter_woba_2018['playerid'] == pid]
    
    PA_2016 = pid_woba_2016['PA_vL'] + pid_woba_2016['PA_vR']
    PA_2017 = pid_woba_2017['PA_vL'] + pid_woba_2017['PA_vR']
    PA_2018 = pid_woba_2018['PA_vL'] + pid_woba_2018['PA_vR']
    
    ISO_vL = pid_woba_2016['ISO_vL'] * 0.05 + pid_woba_2017['ISO_vL'] * 0.25 + pid_woba_2018['ISO_vL'] * 0.7
    ISO_vR = pid_woba_2016['ISO_vR'] * 0.05 + pid_woba_2017['ISO_vR'] * 0.25 + pid_woba_2018['ISO_vR'] * 0.7
    
    wOBA_vL = pid_woba_2016['wRC+_vL'] * 0.05 + pid_woba_2017['wRC+_vL'] * 0.25 + pid_woba_2018['wRC+_vL'] * 0.7
    wOBA_vR = pid_woba_2016['wRC+_vR'] * 0.05 + pid_woba_2017['wRC+_vR'] * 0.25 + pid_woba_2018['wRC+_vR'] * 0.7

    print(pid_woba_2016['PA_vL'])
    
    
    dict_woba['pid'] = pid
    dict_woba['wRC+_vL'] = pid_woba_2018['wRC+_vL']
    dict_woba['wRC+_vR'] = pid_woba_2018['wRC+_vR']
#     print(dict_woba)
    df = df.append(dict_woba, ignore_index=True)
#     result = pd.merge(pid_woba_2016[['Name', 'ISO_vL', 'ISO_vR', 'wRC+_vL', 'wRC+_vR', 'playerid']],
#                       pid_woba_2017[['ISO_vL', 'ISO_vR', 'wRC+_vL', 'wRC+_vR', 'playerid']], 
#                       left_on="playerid", right_on="playerid", suffixes=('_2016', '_2017'))
#     result = pd.merge(result, pid_woba_2018[['ISO_vL', 'ISO_vR', 'wRC+_vL', 'wRC+_vR', 'playerid']], 
#                       left_on="playerid", right_on="playerid")
    break

620    13
Name: PA_vL, dtype: int64


In [82]:
bid_salary_pitcher = pitcher_salary.merge(playeridmap, left_on="Name", right_on="DRAFTKINGSNAME", how="inner")
id_salary_batter = batter_salary.merge(playeridmap, left_on="Name", right_on="DRAFTKINGSNAME", how="inner")

id_salary_pitcher.IDFANGRAPHS = pd.to_numeric(id_salary_pitcher.IDFANGRAPHS, errors='coerce')
id_salary_batter.IDFANGRAPHS = pd.to_numeric(id_salary_batter.IDFANGRAPHS, errors='coerce')

In [60]:
id_salary_pitcher_splits = id_salary_pitcher.merge(pitcher_woba, left_on="IDFANGRAPHS", right_on="playerid", how="inner")
id_salary_batter_splits = id_salary_batter.merge(batter_woba, left_on="IDFANGRAPHS", right_on="playerid", how="inner")

In [61]:
pitcher_splits = id_salary_pitcher_splits[['Position', 'Name_x', 'Salary', 'TeamAbbrev', 'wOBA_x', 'wOBA_y']]
batter_splits = id_salary_batter_splits[['Position', 'Name_x', 'Salary', 'TeamAbbrev', 'wOBA_x', 'wOBA_y']]

In [62]:
pitcher_splits.head()

,Position,Name_x,Salary,TeamAbbrev,wOBA_x,wOBA_y
0,SP,Justin Verlander,11000,HOU,0.252,0.265
1,SP,Corey Kluber,10500,CLE,0.290,0.250
2,SP,Gerrit Cole,10400,HOU,0.236,0.293
3,SP,Clayton Kershaw,10200,LAD,0.290,0.267
4,SP,Trevor Bauer,9900,CLE,0.256,0.264
